In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans
from scipy import stats
from scipy.spatial import distance
import random
random.seed(8888)

In [117]:
# 1. load & filter data 
data0 = pd.read_csv('~/Desktop/data/data02_13_morning_Pages', sep = ',')
#data1 = data0[(((data0.srcAddr == '128.143.2.75') & (data0.srcPort == 80)) | ((data0.dstAddr =='128.143.2.75') & (data0.dstPort == 80)))]
data2 = data0[["bytes","pkts","dur","rate"]]
#len(data2)

In [111]:
# ISO
clf = IsolationForest(max_samples=1000, contamination=0.05,random_state= 5, bootstrap=False)
clf.fit(data2)
y_pred = clf.predict(data2)
data2['ISO'] = y_pred
anomalies = data2[data2.ISO == -1]
len(anomalies)

/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


209

In [118]:
# LOF
clf = LocalOutlierFactor(n_neighbors=20)
y_pred = clf.fit_predict(data2)
data2['LOF'] = y_pred
anomalies = data2[data2.LOF == -1]
len(anomalies)

/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


417

In [119]:
anomalies = anomalies[["bytes","pkts","dur","rate"]]
len(anomalies)

417

In [84]:
# 3. run KSE 
def euc_dis(p1,p2):
    a = p1.values
    b = p2.values
    c = np.vstack((a,b))
    dist = distance.pdist(c)
    return float(dist)
def KSE_Test(df, sample_size):
    # initiate two sample dfs, one distance matrix, and one kse score.
    # generate 2 random samples.
    sample1 = df.sample(n = sample_size, replace = False)
    sample2 = df.sample(n = sample_size, replace = False)
    kse_score = [] # empty list to store KSE scores
#   distance_matrix = np.matrix([[None] * int(len(df) * fraction) , 
#                               [None] * int(len(df) * fraction)])
#     distance_matrix = [[0 for x in range(len(sample1))] 
#                        for y in range(len(sample2))] 
    distance_matrix = np.zeros((sample_size, sample_size))
    # calculate distance from all points in sample1 to all points in sample2
    for i in range(0,sample_size):
        for j in range(0,sample_size):
            distance_matrix[i][j] = euc_dis(sample1.iloc[i,],sample2.iloc[j,])
    print("half way there")
    
    # calculate average KS-score for each instance in df.
    for i in range(0, len(df)):
        observed_instance = df.iloc[i,]
        distance_array = [0] * sample_size
        for j in range(0,sample_size):
            distance_array[j] = euc_dis(observed_instance,sample2.iloc[j,])
        sum = 0
        for k in range(0, sample_size):
            ks_score = stats.ks_2samp(distance_array, distance_matrix[k])[0]
            sum += ks_score
        average = sum/sample_size
        #kse_score[i] = average
        kse_score.append((i, average))
        if (i%1000 == 0):
            print('loop %s' % i)
    return kse_score 


half way there
loop 0


In [120]:
kse_result = KSE_Test(anomalies, int(0.5 * len(anomalies)))


half way there
loop 0


In [89]:
kse_result

[(0, 0.88888888888888884),
 (1, 0.4320987654320988),
 (2, 0.4320987654320988),
 (3, 0.65432098765432078),
 (4, 0.41975308641975312),
 (5, 0.4320987654320988),
 (6, 0.4320987654320988),
 (7, 0.41975308641975312),
 (8, 0.80246913580246926),
 (9, 0.45679012345679015),
 (10, 0.61728395061728392),
 (11, 0.88888888888888884),
 (12, 0.53086419753086433),
 (13, 0.6049382716049384),
 (14, 0.49382716049382719),
 (15, 0.72839506172839519),
 (16, 0.58024691358024705),
 (17, 0.59259259259259245),
 (18, 0.44444444444444442)]

In [121]:
# 4. get result
index_anom = [] 
for i in kse_result:
    if i[1] > 0.5:
        index_anom.append(i[0])
# final result is a data set of the anomalies of the anomalies
final = anomalies.iloc[index_anom]

In [125]:
len(final)

95

In [123]:
final.to_csv('~/Desktop/data/results/Monday_morning_Pages_LOF.csv')

In [124]:
final

,bytes,pkts,dur,rate
116,408336,288,2.642,1236445.14
165,213216,172,3.605,473156.17
196,843032,584,1.580,4268516.66
243,880388,596,0.373,18882316.35
247,911972,616,0.417,17495865.71
412,27148,32,0.444,489153.15
449,549612,380,0.238,18474359.46
474,2694404,1812,0.748,28817149.50
561,1001408,712,2.042,3923243.77
570,628748,440,1.203,4181200.92


In [126]:
pd.read_csv('~/Desktop/data/results/Monday_morning_Pages_LOF.csv')

,Unnamed: 0,bytes,pkts,dur,rate
0,116,408336,288,2.642,1236445.14
1,165,213216,172,3.605,473156.17
2,196,843032,584,1.580,4268516.66
3,243,880388,596,0.373,18882316.35
4,247,911972,616,0.417,17495865.71
5,412,27148,32,0.444,489153.15
6,449,549612,380,0.238,18474359.46
7,474,2694404,1812,0.748,28817149.50
8,561,1001408,712,2.042,3923243.77
9,570,628748,440,1.203,4181200.92
